<a href="https://colab.research.google.com/github/tsato-code/colab_notebooks/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [20]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [21]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [22]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [23]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [24]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [25]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [26]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 01s]
val_accuracy: 0.8540999889373779

Best val_accuracy So Far: 0.885699987411499
Total elapsed time: 00h 12m 06s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



In [27]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.5741 - accuracy: 0.8001 - val_loss: 0.4257 - val_accuracy: 0.8444
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3685 - accuracy: 0.8641 - val_loss: 0.4476 - val_accuracy: 0.8380
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3250 - accuracy: 0.8818 - val_loss: 0.3556 - val_accuracy: 0.8706
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2951 - accuracy: 0.8904 - val_loss: 0.3653 - val_accuracy: 0.8686
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2768 - accuracy: 0.8969 - val_loss: 0.3672 - val_accuracy: 0.8593
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2666 - accuracy: 0.9008 - val_loss: 0.3349 - val_accuracy: 0.8857
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2560 - accuracy: 0.9039 - val_loss: 0.3405 - val_accuracy:

In [29]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 448
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.885699987411499
Trial summary
Hyperparameters:
units: 384
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 01fe580fb7918ed5bed74393d6307d09
Score: 0.8841999769210815
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.879800021648407
Trial summary
Hyperparameters:
units: 320
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: c1e79d874ac7bb33ff7c3bbbd4f03331
Score: 0.8779000043869019
Trial summary
Hyperparameters:
units: 448
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: